In [ ]:
import inspect

import DA_methods
import matplotlib.pyplot as plt
import matplotlib.tri
import numpy as np
from L96_model import L96, L96_eq1_xdot

This notebook is a stripped-down copy of the notebook in `03Data-Assimilation` directory and altered the parameters of the Lorenz 1996 model to match those of Wilks, 2005.
- For the purposes of this illustration, we removed all the unused options and parameters in the DA algorithm.
- The L96 model here uses K=8, J=32, F=18, as in Wilks, 2005.

In [ ]:
# For reproducibility
rng = np.random.default_rng(3210)

In [ ]:
# Define a function to represent the GCM, and many other convenience functions used within the DA algorithm


def GCM(X0, F, dt, nt, param=[0]):
    time, hist, X = (
        dt * np.arange(nt + 1),
        np.zeros((nt + 1, len(X0))) * np.nan,
        X0.copy(),
    )
    hist[0] = X

    for n in range(nt):
        X = X + dt * (L96_eq1_xdot(X, F) - np.polyval(param, X))

        hist[n + 1], time[n + 1] = X, dt * (n + 1)
    return hist, time


def s(k, K):
    """A non-dimension coordinate from -1..+1 corresponding to k=0..K"""
    return 2 * (0.5 + k) / K - 1


def get_dist(i, j, K):
    return abs(i - j) if abs(i - j) <= 0.5 * K / 2 else K - abs(i - j)


# Generate observation operator, assuming linearity and model space observations
def ObsOp(K, l_obs, t_obs, i_t):
    nobs = l_obs.shape[-1]
    H = np.zeros((nobs, K))
    H[range(nobs), l_obs[t_obs == i_t]] = 1
    return H


# localize covariance matrix based on the Gaspari-Cohn function
def cov_loc(B, loc=0):
    M, N = B.shape
    X, Y = np.ix_(np.arange(M), np.arange(N))
    dist = np.vectorize(get_dist)(X, Y, M)
    W = np.vectorize(gaspari_cohn)(dist, loc)
    return B * W, W


def gaspari_cohn(distance, radius):
    if distance == 0:
        weight = 1.0
    else:
        if radius == 0:
            weight = 0.0
        else:
            ratio = abs(distance / radius)
            weight = 0.0
            if ratio <= 1:
                weight = (
                    -(ratio**5) / 4
                    + ratio**4 / 2
                    + 5 * ratio**3 / 8
                    - 5 * ratio**2 / 3
                    + 1
                )
            elif ratio <= 2:
                weight = (
                    ratio**5 / 12
                    - ratio**4 / 2
                    + 5 * ratio**3 / 8
                    + 5 * ratio**2 / 3
                    - 5 * ratio
                    + 4
                    - 2 / 3 / ratio
                )
    return weight


def find_obs(loc, obs, t_obs, l_obs, period):
    t_period = np.where((t_obs[:, 0] >= period[0]) & (t_obs[:, 0] < period[1]))
    obs_period = np.zeros(t_period[0].shape)
    obs_period[:] = np.nan
    for i in np.arange(len(obs_period)):
        if np.any(l_obs[t_period[0][i]] == loc):
            obs_period[i] = obs[t_period[0][i]][l_obs[t_period[0][i]] == loc]
    return obs_period


def running_ave(X, N):
    if N % 2 == 0:
        N1, N2 = -N / 2, N / 2
    else:
        N1, N2 = -(N - 1) / 2, (N + 1) / 2

    X_sum = np.zeros(X.shape)
    for i in np.arange(N1, N2):
        X_sum = X_sum + np.roll(X, int(i), axis=0)
    return X_sum / N

In [ ]:
# Goldilocks settings
config = dict(
    K=8,  # Dimension of L96 "X" variables
    J=32,  # Dimension of L96 "Y" variables
    obs_freq=10,  # observation frequency (number of sampling intervals (si) per observation)
    F_truth=18,  # F for truth signal
    F_fcst=18,  # F for forecast (DA) model
    GCM_param=np.array(
        [0, 0, 0, 0]
    ),  # polynomial coefficicents for GCM parameterization
    ns_da=4000,  # number of time samples for DA
    ns=4000,  # number of time samples for truth signal
    ns_spinup=200,  # number of time samples for spin up
    dt=0.005,  # model timestep
    si=0.005,  # truth sampling interval
    B_loc=0.0,  # spatial localization radius for DA
    DA="EnKF",  # DA method
    nens=50,  # number of ensemble members for DA
    inflate_opt="relaxation",  # method for DA model covariance inflation
    inflate_factor=0.86,  # inflation factor
    obs_density=1.0,  # fraction of spatial gridpoints where observations are collected
    DA_freq=10,  # assimilation frequency (number of sampling intervals (si) per assimilation step)
    obs_sigma=0.1,  # observational error standard deviation
    initial_spread=0.1,  # Initial spread added to initial conditions
)

# Uncomment blocks below to perturb the above settings

# # Less certain observations
# config['obs_sigma'] = 1.0
# config['initial_spread'] = 1.0
# config['inflate_factor'] = 0.5

# # Less frequent observations
# config['obs_freq'] = 50
# config['DA_freq'] = 50
# config['inflate_factor'] = 0.4

# # Very infrequent observations
# config['obs_freq'] = 200
# config['DA_freq'] = 200
# config['inflate_factor'] = 0.5

# # More frequent observations
# config['obs_freq'] = 5
# config['DA_freq'] = 5
# config['inflate_factor'] = 0.9

# # Very frequent observations
# config['obs_freq'] = 1
# config['DA_freq'] = 1
# config['inflate_factor'] = 0.98

# # Very frequent observations but less accurate
# config['obs_freq'] = 1
# config['DA_freq'] = 1
# config['obs_sigma'] = 1.0
# config['initial_spread'] = 1.0
# config['inflate_factor'] = 0.9

# # Different time-scale
# config['F_fcst'] = 16

The 'real world" is the Lorenz '96 model:
\begin{align}
\frac{d}{dt} X_k
&= - X_{k-1} \left( X_{k-2} - X_{k+1} \right) - X_k + F - \left( \frac{hc}{b} \right) \sum_{j=0}^{J-1} Y_{j,k}
\\
\frac{d}{dt} Y_{j,k}
&= - cbY_{j+1,k} \left( Y_{j+2,k} - X_{j-1,k} \right) - c Y_{j,k} + \frac{hc}{b} X_k
\end{align}

The cell below spins-up a state and then records a series of $X$ and $Y$ at time $t$ in arrays `X_truth`, `Y_truth` and `t_truth` respectively. The initial state $X(t=0$ is recorded in `X_init` (and equal to `X_truth[0]`).

In [ ]:
# Set up the "truth" 2-scale L96 model and generate initial conditions from a short spinup
M_truth = L96(config["K"], config["J"], F=config["F_truth"], dt=config["dt"])
M_truth.set_state(rng.standard_normal((config["K"])), 0 * M_truth.j)
X_init, Y_init, _ = M_truth.run(config["si"], config["si"] * config["ns_spinup"])
X_init, Y_init = X_init[-1, :], Y_init[-1, :]
M_truth.set_state(X_init, Y_init)

# Run L96 to generate the "truth"
X_truth, Y_truth, t_truth = M_truth.run(config["si"], config["si"] * config["ns"])

del Y_init  # Not used below

Now we create some "observations" of the "real world" by sampling at `obs_freq` intervals and adding some noise (observational error). `X_obs` are the observations at `k=l_obs` positions and `t=t_truth[t_obs]` times. Note that `t_obs` is an index.

In [ ]:
# Sample the "truth" to generate observations at certain times (t_obs) and locations (l_obs)
t_obs = np.tile(
    config["obs_freq"] + np.arange(0, config["ns_da"], config["obs_freq"]),
    [int(config["K"] * config["obs_density"]), 1],
).T
l_obs = np.zeros(t_obs.shape, dtype="int")
for i in range(l_obs.shape[0]):
    l_obs[i, :] = rng.choice(
        config["K"], int(config["K"] * config["obs_density"]), replace=False
    )
X_obs = X_truth[t_obs, l_obs] + config["obs_sigma"] * rng.standard_normal(l_obs.shape)

# Calculated observation covariance matrix, assuming independent observations
R = config["obs_sigma"] ** 2 * np.eye(int(config["K"] * config["obs_density"]))

plt.figure(figsize=[10, 6])
plt.plot(t_truth[:], X_truth[:, 0], label="truth")
plt.scatter(
    t_truth[t_obs[1:, 0]],
    find_obs(0, X_obs, t_obs, l_obs, [t_obs[0, 0], t_obs[-1, 0]]),
    color="k",
    label="obs",
)
plt.legend()
plt.xlabel("Time t")
plt.ylabel("X(t)")
plt.title("Observations at k=0");

We run the model in forward mode for 5000 steps to calculate the background covariance. The model is the "GCM" function defined above which integrates forward
\begin{align}
\frac{d}{dt} X_k
&= - X_{k-1} \left( X_{k-2} - X_{k+1} \right) - X_k + F
\end{align}
The absence of the coupling term to the $Y$ equations makes this a model with "missing physics" that we hope the Ensemble Kalman Filter will correct.

In [ ]:
# generate climatological background covariance for 1-scale L96 model
X1_clim, _ = GCM(X_init, config["F_fcst"], config["dt"], 5000)
B_clim1 = np.cov(X1_clim.T)
del X1_clim

# load pre-calculated climatological background covariance matrix from a long simulation
# B_clim1=np.load('B_clim_L96s.npy')
B_loc, W_clim = cov_loc(B_clim1, loc=config["B_loc"])

B_corr1 = np.zeros(B_clim1.shape)
for i in range(B_clim1.shape[0]):
    for j in range(B_clim1.shape[1]):
        B_corr1[i, j] = B_clim1[i, j] / np.sqrt(B_clim1[i, i] * B_clim1[j, j])

plt.figure(figsize=(16, 6))
plt.subplot(131)
plt.contourf(B_corr1, cmap="bwr", extend="both", levels=np.linspace(-0.95, 0.95, 20))
plt.colorbar()
plt.title("Background correlation matrix 1-scale L96")
plt.subplot(132)
plt.contourf(B_loc)
plt.colorbar()
plt.title("B_loc")
plt.subplot(133)
plt.contourf(W_clim)
plt.colorbar()
plt.title("W_clim")

del B_loc  # not used below

This is the actual DA algorithm. It steps through segments of time ("DA cycles"), launching an ensemble of short forecasts from the posterior estimate of the preceding segment, each perturbed by noise in their initial condition (inflation).

Each ensemble trajectory is stored in `ensX`. The increment added to correct the prior is in `X_inc`.

In [ ]:
# set up array to store DA increments
X_inc = np.zeros(
    (int(config["ns_da"] / config["DA_freq"]), config["K"], config["nens"])
)
if config["DA"] == "3DVar":
    X_inc = np.squeeze(X_inc)
t_DA = np.zeros(int(config["ns_da"] / config["DA_freq"]))

# initialize ensemble with perturbations
i_t = 0
ensX = (
    X_init[None, :, None]
    + rng.standard_normal((1, config["K"], config["nens"])) * config["initial_spread"]
)
X_post = ensX[0, ...]

W = W_clim

# DA cycles
for cycle in np.arange(0, config["ns_da"] / config["DA_freq"], dtype="int"):

    # set up array to store model forecast for each DA cycle
    ensX_fcst = np.zeros((config["DA_freq"] + 1, config["K"], config["nens"]))

    # model forecast for next DA cycle
    for n in range(config["nens"]):
        ensX_fcst[..., n] = GCM(
            X_post[0 : config["K"], n],
            config["F_fcst"],
            config["dt"],
            config["DA_freq"],
            config["GCM_param"],
        )[0]
    i_t = i_t + config["DA_freq"]

    # get prior/background from the forecast
    X_prior = ensX_fcst[-1, ...]  # get prior from model integration

    # call DA
    t_DA[cycle] = t_truth[i_t]
    if config["DA"] == "EnKF":
        H = ObsOp(config["K"], l_obs, t_obs, i_t)
        # augment state vector with parameters when doing parameter estimation
        B_ens = np.cov(X_prior)
        B_ens_loc = B_ens * W[0 : config["K"], 0 : config["K"]]
        X_post = DA_methods.EnKF(X_prior, X_obs[t_obs == i_t], H, R, B_ens_loc)
        X_post[0 : config["K"], :] = DA_methods.ens_inflate(
            X_post[0 : config["K"], :],
            X_prior[0 : config["K"], :],
            config["inflate_opt"],
            config["inflate_factor"],
        )
    elif config["DA"] == "None":
        X_post = X_prior

    if not config["DA"] == "None":
        X_inc[cycle, ...] = (
            np.squeeze(X_post[0 : config["K"], ...]) - X_prior[0 : config["K"], ...]
        )  # get current increments
        # get posterior info about the estimated parameters

    # reset initial conditions for next DA cycle
    ensX_fcst[-1, :, :] = X_post[0 : config["K"], :]
    ensX = np.concatenate((ensX, ensX_fcst[1:None, ...]))

`meanX` is the ensemble mean forecast, averaging over all the ensemble members. It has discontinuities due to the increment add between each DA segment.

In [ ]:
# post processing and visualization
meanX = np.mean(ensX, axis=-1)
clim = np.max(np.abs(meanX - X_truth[0 : (config["ns_da"] + 1), :]))

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
ch = axes[0, 0].contourf(
    M_truth.k,
    t_truth[0 : (config["ns_da"] + 1)],
    meanX - X_truth[0 : (config["ns_da"] + 1), :],
    cmap="bwr",
    levels=np.arange(-6.5, 7, 1),
    extend="both",
)
plt.colorbar(ch, ax=axes[0, 0], orientation="horizontal")
axes[0, 0].set_xlabel("s")
axes[0, 0].set_ylabel("t")
axes[0, 0].set_title("X - X_truth")
axes[0, 1].plot(
    t_truth[0 : (config["ns_da"] + 1)],
    np.sqrt(((meanX - X_truth[0 : (config["ns_da"] + 1), :]) ** 2).mean(axis=-1)),
    label="RMSE",
)
axes[0, 1].plot(
    t_truth[0 : (config["ns_da"] + 1)],
    np.mean(np.std(ensX, axis=-1), axis=-1),
    label="Spread",
)
axes[0, 1].plot(
    t_truth[0 : (config["ns_da"] + 1)],
    config["obs_sigma"] * np.ones((config["ns_da"] + 1)),
    label="Obs error",
)
axes[0, 1].legend()
axes[0, 1].set_xlabel("time")
axes[0, 1].set_title("RMSE (X - X_truth)")
axes[0, 1].grid(which="both", linestyle="--")

axes[0, 2].plot(
    M_truth.k,
    np.sqrt(((meanX - X_truth[0 : (config["ns_da"] + 1), :]) ** 2).mean(axis=0)),
    label="RMSE",
)
X_inc_ave = X_inc / config["DA_freq"] / config["si"]
axes[0, 2].plot(M_truth.k, X_inc_ave.mean(axis=(0, -1)), label="Inc")
axes[0, 2].plot(
    M_truth.k, running_ave(X_inc_ave.mean(axis=(0, -1)), 7), label="Inc Ave"
)
axes[0, 2].plot(
    M_truth.k,
    np.ones(M_truth.k.shape) * (config["F_fcst"] - config["F_truth"]),
    label="F_bias",
)
axes[0, 2].plot(
    M_truth.k,
    np.ones(M_truth.k.shape) * (X_inc / config["DA_freq"] / config["si"]).mean(),
    "k:",
    label="Ave Inc",
)
axes[0, 2].legend()
axes[0, 2].set_xlabel("s")
axes[0, 2].set_title("Increments")
axes[0, 2].grid(which="both", linestyle="--")

# X_inc_ave=(X_inc/config['DA_freq']/config['si']).mean(axis=(1,2)).\
#         reshape(int(config['ns_da']/ann_period),int(ann_period/config['DA_freq'])).mean(axis=0)
# axes[0,2].plot(np.arange(ann_period/config['DA_freq']),X_inc_ave,label='Inc')
# axes[0,2].plot(np.arange(ann_period/config['DA_freq']),running_ave(X_inc_ave,10),label='Inc Ave');
# axes[0,2].plot(np.arange(0,ann_period/config['DA_freq'],mon_period/config['DA_freq']),
#                -2*np.sin(2*np.pi*np.arange(mon_per_ann)/mon_per_ann),label='F_bias')
# axes[0,2].legend()
# axes[0,2].set_xlabel('"annual cycle"'); axes[0,2].set_title('Increments');
# axes[0,2].grid(which='both',linestyle='--')

plot_start, plot_end = 200, 800
plot_start_DA, plot_end_DA = int(plot_start / config["DA_freq"]), int(
    plot_end / config["DA_freq"]
)
plot_x = 0
axes[1, 0].plot(
    t_truth[plot_start:plot_end], X_truth[plot_start:plot_end, plot_x], label="truth"
)
axes[1, 0].plot(
    t_truth[plot_start:plot_end], meanX[plot_start:plot_end, plot_x], label="forecast"
)
axes[1, 0].scatter(
    t_DA[plot_start_DA - 1 : plot_end_DA - 1],
    find_obs(plot_x, X_obs, t_obs, l_obs, [plot_start, plot_end]),
    label="obs",
)
axes[1, 0].grid(which="both", linestyle="--")
axes[1, 0].set_xlabel("time")
axes[1, 0].set_title("k=" + str(plot_x + 1) + " truth and forecast")
axes[1, 0].legend()

axes[1, 2].text(
    0.1,
    0.1,
    "RMSE={:3f}\nSpread={:3f}\nGCM param={}\nDA={},{}\nDA_freq={}\nB_loc={}\ninflation={},{}\nobs_density={}\nobs_sigma={}\nobs_freq={}".format(
        np.sqrt(((meanX - X_truth[0 : (config["ns_da"] + 1), :]) ** 2).mean()),
        np.mean(np.std(ensX, axis=-1)),
        config["DA"],
        config["GCM_param"],
        config["nens"],
        config["DA_freq"],
        config["B_loc"],
        config["inflate_opt"],
        config["inflate_factor"],
        config["obs_density"],
        config["obs_sigma"],
        config["obs_freq"],
    ),
    fontsize=15,
);

Converting the increment `X_inc` into a tendency, we can examine the relationship between the $\dot{X}$ due to the missing physics and the state of the model at the beginning of each DA segment. If we are properly correcting the absence of the coupling term then this structure should look like the parameterization of the coupling term, as done in Wilks, 2005.

In [ ]:
jj = np.abs(X_inc_ave[0:, :].flatten()) > -1e-7

x_input = ensX[t_obs[0:, 0] - config["DA_freq"], :].flatten()[
    jj
]  # The offset by DA_freq looks at the previous posterior
x_input = 0.5 * (
    x_input + ensX[t_obs[0:, 0], :].flatten()[jj]
)  # Mid-point of trajectory
xinc_output = X_inc_ave[0:, :].flatten()[jj]

x = np.linspace(-8, 15, 100)
p = np.polyfit(x_input, xinc_output, 4)
p18 = [0.000707, -0.0130, -0.0190, 1.59, 0.275]  # Polynomial from Wilks, 2005

plt.figure(figsize=(12, 5))
plt.suptitle("All time, all individial k, and all ensemble members")
plt.subplot(121)
plt.plot(x_input, xinc_output, "k.")
plt.grid()
plt.plot(x, -np.polyval(p18, x), label="$P_4(X_k)$ - Wilks, 2005")
plt.plot(x, np.polyval(p, x), label="$P_4(X_k)$")
plt.xlabel("Ensemble member $X_i(k,t)$")
plt.ylabel("Ensemble member increment $\dot{X}$")
plt.subplot(122)
plt.hist2d(
    x_input, xinc_output, bins=(np.linspace(-10, 15, 50), np.linspace(-25, 20, 150))
)
plt.plot(x, -np.polyval(p18, x), label="$P_4(X_k)$ - Wilks, 2005")
plt.plot(x, np.polyval(p, x), label="$P_4(X_k)$")
plt.xlabel("Ensemble member $X_i(k,t)$")
plt.ylabel("Ensemble member increment $\dot{X}$");

In [ ]:
jj = np.abs(X_inc_ave.mean(axis=-1).flatten()) > -1e-7

x_input = meanX[t_obs[0:, 0] - config["DA_freq"]].flatten()[
    jj
]  # The offset by DA_freq looks at the previous posterior
x_input = 0.5 * (x_input + meanX[t_obs[0:, 0]].flatten()[jj])  # Mid-point of trajectory
xinc_output = X_inc_ave.mean(axis=-1).flatten()[jj]

x = np.linspace(-8, 15, 100)
p = np.polyfit(x_input, xinc_output, 4)
p18 = [0.000707, -0.0130, -0.0190, 1.59, 0.275]  # Polynomial from Wilks, 2005

plt.figure(figsize=(12, 5))
plt.suptitle("All time, all individial k, mean over ensemble members")
plt.subplot(121)
plt.plot(x_input, xinc_output, "k.")
plt.grid()
plt.plot(x, -np.polyval(p18, x), label="$P_4(X_k)$ - Wilks, 2005")
plt.plot(x, np.polyval(p, x), label="$P_4(X_k)")
plt.xlabel("Ensemble member $X_i(k,t)$")
plt.ylabel("Ensemble member increment $\dot{X}$")
plt.subplot(122)
plt.hist2d(
    x_input, xinc_output, bins=(np.linspace(-10, 15, 50), np.linspace(-25, 20, 150))
)
plt.plot(x, -np.polyval(p18, x), label="$P_4(X_k)$ - Wilks, 2005")
plt.plot(x, np.polyval(p, x), label="$P_4(X_k)")
plt.xlabel("Ensemble member $X_i(k,t)$")
plt.ylabel("Ensemble member increment $\dot{X}$");

In [ ]:
xl = 8, 10
k = 0
si = config["DA_freq"]

l = (l_obs == k).max(axis=1)  # True if observation at time t_obs for column k

plt.figure(figsize=(14, 5))
plt.suptitle("Ensemble mean, k = %i" % (k))
plt.plot(t_truth, X_truth[:, k], "--", label="Truth")
plt.fill_between(
    t_truth,
    meanX[:, k] - ensX[:, k, :].std(axis=-1),
    meanX[:, k] + ensX[:, k, :].std(axis=-1),
    color="grey",
    alpha=0.2,
    label="Ensemble spread",
)
plt.plot(t_truth, meanX[:, k], label="Ensemble mean forecast")
plt.plot(
    t_truth[t_obs[l, 0]],
    meanX[si::si, k][l] - X_inc.mean(axis=-1)[l, k],
    ".",
    label="Ensemble mean prior",
)
plt.plot(t_truth[t_obs[l, 0]], meanX[si::si, k][l], ".", label="Ensemble mean post")
plt.xlim(xl)
plt.xlabel("Time, t")
plt.ylabel("$X(t)$")
plt.legend();

In [ ]:
xl = 8, 10
k = 0
e = 19
si = config["DA_freq"]

l = (l_obs == k).max(axis=1)  # True if observation at time t_obs for column k

plt.figure(figsize=(14, 5))
plt.suptitle("Ensemble member %i , k = %i" % (e, k))
plt.plot(t_truth, X_truth[:, k], "--", label="Truth")
plt.plot(t_truth, ensX[:, k, e], label="Ensemble member forecast")
plt.plot(
    t_truth[t_obs[l, 0]],
    ensX[si::si, k, e][l] - X_inc[:, :, e][l, k],
    ".",
    label="Ensemble member prior",
)
plt.plot(t_truth[t_obs[l, 0]], ensX[si::si, k, e][l], ".", label="Ensemble member post")
plt.xlim(xl)
plt.xlabel("Time, t")
plt.ylabel("$X(t)$")
plt.legend();

In [ ]:
# Save data for other notebooks to read
np.savez(
    "increments.npz",
    ensX=ensX,
    X_inc=X_inc,
    t_obs=t_obs,
    l_obs=l_obs,
    t_inc=t_truth[t_obs[:, 0]],
    X_truth=X_truth,
    Y_truth=Y_truth,
    t_truth=t_truth,
    X_obs=X_obs,
)